In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data=pd.read_csv('0000000000002747_training_twitter_x_y_train.csv')
train_data.head()

In [ ]:
x_training = train_data[['text','airline_sentiment']]

In [ ]:
test_data=pd.read_csv('0000000000002747_test_twitter_x_test.csv')
test_data.head()

In [ ]:
x_testing = test_data['text']
x_test = x_testing.values

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree

In [ ]:
type(x_training)

In [ ]:
x_train = x_training.values

In [ ]:
train_words = []
for i in range(len(x_train)):
    train_words.append([word_tokenize(x_train[i][0]), x_train[i][1]])

In [ ]:
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
stops

In [ ]:
from nltk.corpus import wordnet
def get_simple_pos_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
lemmatizer = WordNetLemmatizer()
def clean(words):
    output_words = []
    for w in words:
        if w.isalpha():
            if(w.lower() not in stops):
                pos = pos_tag([w])
                clean_words = lemmatizer.lemmatize(w, pos=get_simple_pos_tag(pos[0][1]))
                output_words.append(clean_words.lower())
    return output_words

In [ ]:
for i in range(len(train_words)):
    train_words[i] = (clean(train_words[i][0]), train_words[i][1]) 

In [ ]:
y_train = []
tweet = []
for tweets, res in train_words:
    tweet.append(" ".join(tweets))
    y_train.append(res)

In [ ]:
tweets_test = []
for i in x_test:
    i = clean(word_tokenize(i))
    tweets_test.append(" ".join(i))

In [ ]:
tweets_test

Count_vectorizer

In [ ]:
count_vec = CountVectorizer(max_features=10000, ngram_range = (2,3)) 
x_train_features = count_vec.fit_transform(tweet)

In [ ]:
x_test_features = count_vec.transform(tweets_test)

Perform classifications

In [ ]:
clf1 = SVC()
clf1.fit(x_train_features, y_train)

In [ ]:
y_predict_svm = clf1.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_predict_svm)
df.to_csv('svm_more_predictions.csv', index = False, header = False)

In [ ]:
clf2 = RandomForestClassifier()
clf2.fit(x_train_features, y_train)

In [ ]:
y_predict_rf=clf2.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_predict_rf)
df.to_csv('rf_predictions.csv', index = False, header = False)

In [ ]:
clf3 = MultinomialNB()
clf3.fit(x_train_features, y_train)

In [ ]:
y_predict_mnb = clf3.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_predict_mnb)
df.to_csv('mnb_predictions.csv', index = False, header = False)

In [ ]:
clf4 = tree.DecisionTreeClassifier()
clf4.fit(x_train_features, y_train)

In [ ]:
y_predict_dt = clf4.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_predict_dt)
df.to_csv('dt_predictions.csv', index = False, header = False)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf5 = KNeighborsClassifier()
clf5.fit(x_train_features, y_train)

In [ ]:
y_predict_KNN = clf5.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_predict_KNN)
df.to_csv('KNN_predictions.csv', index = False, header = False)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf6 = LogisticRegression()
clf6.fit(x_train_features, y_train)

In [ ]:
y_predict_LG = clf6.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_predict_LG)
df.to_csv('LG_more_predictions.csv', index = False, header = False)

In [ ]:
clf7 = MultinomialNB(alpha=1)
clf7.fit(x_train_features, y_train)

In [ ]:
y_predict_mnb1 = clf3.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_predict_mnb1)
df.to_csv('mnb1_predictions.csv', index = False, header = False)

In [ ]:
from sklearn.model_selection import GridSearchCV
clf = SVC()
grid = {'C':[1e1,1e2,1e3,1e4,5e3,5e2],'gamma':[1e-3,1e-2,5e-3,1e-4,5e-2,1e-5]}
abc = GridSearchCV(clf,grid,n_jobs=-1)
abc.fit(x_train_features, y_train)

In [ ]:
y_pred = abc.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_pred)
df.to_csv('svc_updated_predictions.csv', index = False, header = False)

In [ ]:
abc.best_params_

In [ ]:
lr = LogisticRegression()
grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}
model_lr = GridSearchCV(lr, param_grid=grid_values)

In [ ]:
model_lr.fit(x_train_features, y_train)

In [ ]:
y_pred1 = model_lr.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_pred1)
df.to_csv('LR_updated_predictions.csv', index = False, header = False)

In [ ]:
model=MultinomialNB()
param={'alpha': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]}
clfmb=GridSearchCV(model,param,scoring='roc_auc',cv=10,return_train_score=True)  
clfmb.fit(x_train_features,y_train)

In [ ]:
y_pred2 = clfmb.predict(x_test_features)

In [ ]:
df = pd.DataFrame(y_pred2)
df.to_csv('MB_updated_predictions.csv', index = False, header = False)